In [3]:
import pandas as pd
import csv
import json
import os
from sklearn.preprocessing import OneHotEncoder
from pandas.core.frame import DataFrame
from sklearn.model_selection import train_test_split
import numpy as np

In [4]:
weathers = pd.read_csv('./Weather/kiruna 2021-01-01 to 2021-12-31.csv')
weathers = weathers[12:8760]
weathers = weathers.reset_index(drop=True)

In [5]:
weather_icon = weathers['icon'] # get weather icons
print(set(weather_icon))
weather_icon = DataFrame(weather_icon) # turn the data type into DataFrame
weather_conditions = weathers['conditions'] # get weather conditions
print(set(weather_conditions))
weather_conditions = DataFrame(weather_conditions) # turn the data type into DataFrame
weather_cloudcover = weathers['cloudcover'] # get weather cloudcover
weather_cloudcover = DataFrame(weather_cloudcover) # turn the data type into DataFrame
weather_visibility = weathers['visibility'] # get weather visibility
weather_visibility = DataFrame(weather_visibility) # turn the data type into DataFrame

{'cloudy', 'clear-night', 'fog', 'wind', 'partly-cloudy-night', 'clear-day', 'partly-cloudy-day', 'snow', 'rain'}
{'Snow, Rain', 'Snow, Partially cloudy', 'Snow, Rain, Overcast', 'Snow', 'Rain, Partially cloudy', 'Partially cloudy', 'Snow, Overcast', 'Rain', 'Clear', 'Overcast', 'Rain, Overcast', 'Snow, Rain, Partially cloudy'}


In [6]:
encoder = OneHotEncoder(handle_unknown='ignore')
weather_icon_onehot = pd.get_dummies(weather_icon['icon'])
weather_conditions_onehot = pd.get_dummies(weather_conditions['conditions'])

In [7]:
weather_icon_onehot

,clear-day,clear-night,cloudy,fog,partly-cloudy-day,partly-cloudy-night,rain,snow,wind
0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...
8743,1,0,0,0,0,0,0,0,0
8744,1,0,0,0,0,0,0,0,0
8745,1,0,0,0,0,0,0,0,0
8746,1,0,0,0,0,0,0,0,0


In [8]:
weather_conditions_onehot

,Clear,Overcast,Partially cloudy,Rain,"Rain, Overcast","Rain, Partially cloudy",Snow,"Snow, Overcast","Snow, Partially cloudy","Snow, Rain","Snow, Rain, Overcast","Snow, Rain, Partially cloudy"
0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8743,1,0,0,0,0,0,0,0,0,0,0,0
8744,1,0,0,0,0,0,0,0,0,0,0,0
8745,1,0,0,0,0,0,0,0,0,0,0,0
8746,1,0,0,0,0,0,0,0,0,0,0,0


In [9]:
with open('./Kp_index/kp.gfz-potsdam.de.json') as f:
    df_Kp = json.load(f)
Kp_index = df_Kp['Kp']
Kp_index = Kp_index[4:2920]
Kp_index.append(2.333) # add the Kp-index of 2021-12-31-24 (2022-01-01-0)
Kp = []
for i in range(0,len(Kp_index)):
    Kp.append(Kp_index[i])
    Kp.append(Kp_index[i])
    Kp.append(Kp_index[i])
Kp = Kp[1:len(Kp)-2]
Kp = DataFrame(Kp)
Kp = Kp.rename(columns={0:'Kp_index'})
Kp

,Kp_index
0,0.333
1,0.333
2,1.333
3,1.333
4,1.333
...,...
8743,0.667
8744,1.000
8745,1.000
8746,1.000


In [10]:
with open('./Aurora_pic/Aurora.json') as f:
    df_Labels = json.load(f)
Labels = df_Labels
Labels = Labels[0:8748]
Labels = DataFrame(Labels)
Labels = Labels.rename(columns={0:'Aurora_Label'})
Labels


,Aurora_Label
0,0
1,0
2,0
3,0
4,0
...,...
8743,0
8744,0
8745,0
8746,0


In [11]:
df_Features = pd.concat([Kp, weather_cloudcover, weather_visibility, weather_icon_onehot, weather_conditions_onehot], axis=1)
df_Features = df_Features.drop(index=[1206, 3429]) # drop nan
df_Features = df_Features.reset_index(drop=True)
Labels = Labels.drop(index=[1206, 3429])
Labels = Labels.reset_index(drop=True)


In [12]:
df_Features_no_onehot = pd.concat([Kp, weather_cloudcover, weather_visibility, weather_icon, weather_conditions], axis=1)
df_Features_no_onehot = df_Features.drop(index=[1206, 3429]) # drop nan
df_Features_no_onehot = df_Features.reset_index(drop=True)


In [13]:
Feature_train,Feature_test,Label_train,Label_test = train_test_split(df_Features,Labels,test_size=0.3)
Feature_no_onehot_train,Feature_no_onehot_test,Label_no_onehot_train,Label_no_onehot_test = train_test_split(df_Features,Labels,test_size=0.3)


In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
def KNN(Feature_train,Label_train,Feature_test):
    model = KNeighborsClassifier(n_neighbors=5)#默认为5
    model.fit(Feature_train,Label_train)
    predicted = model.predict(Feature_test)
    return predicted
Label_pred_KNN = KNN(Feature_no_onehot_train,Label_no_onehot_train.values.ravel(),Feature_no_onehot_test)
print(accuracy_score(Label_no_onehot_test, Label_pred_KNN))
print(roc_auc_score(Label_no_onehot_test, Label_pred_KNN))

0.9409298780487805
0.574139046079224


In [15]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
def KNN(Feature_train,Label_train,Feature_test):
    model = KNeighborsClassifier(n_neighbors=5)#默认为5
    model.fit(Feature_train,Label_train)
    predicted = model.predict(Feature_test)
    return predicted
Label_pred_KNN = KNN(Feature_train,Label_train.values.ravel(),Feature_test)
print(Label_test)
print(Label_pred_KNN)
print(accuracy_score(Label_test, Label_pred_KNN))
print(roc_auc_score(Label_test, Label_pred_KNN))

      Aurora_Label
6092             0
4743             0
6949             1
8036             0
339              0
...            ...
8492             0
6547             0
2171             0
1787             0
1776             0

[2624 rows x 1 columns]
[0 0 0 ... 0 0 0]
0.9416920731707317
0.5384179277710816


In [16]:
from sklearn.tree import DecisionTreeClassifier
def CTRA(X,y,XX):
    model = DecisionTreeClassifier()
    model.fit(X,y)
    predicted = model.predict(XX)
    return predicted
Label_pred_Decision_Tree = CTRA(Feature_no_onehot_train,Label_no_onehot_train,Feature_no_onehot_test)
print(accuracy_score(Label_no_onehot_test, Label_pred_Decision_Tree))
print(roc_auc_score(Label_no_onehot_test, Label_pred_Decision_Tree))

0.9275914634146342
0.6140339531123686


In [17]:
from sklearn.tree import DecisionTreeClassifier
def CTRA(X,y,XX):
    model = DecisionTreeClassifier()
    model.fit(X,y)
    predicted = model.predict(XX)
    return predicted
Label_pred_Decision_Tree = CTRA(Feature_train,Label_train,Feature_test)
print(accuracy_score(Label_test, Label_pred_Decision_Tree))
print(roc_auc_score(Label_test, Label_pred_Decision_Tree))

0.926829268292683
0.5726172938708094


In [18]:
from sklearn.ensemble import RandomForestClassifier
def RFC(X,y,XX):
    model = RandomForestClassifier()
    model.fit(X,y)
    predicted = model.predict(XX)
    return predicted
Label_pred_RandomForest = RFC(Feature_train,Label_train.values.ravel(),Feature_test)
print(accuracy_score(Label_test, Label_pred_RandomForest))
print(roc_auc_score(Label_test, Label_pred_RandomForest))


0.9405487804878049
0.558832194991295


In [19]:
from sklearn.ensemble import RandomForestClassifier
def RFC(X,y,XX):
    model = RandomForestClassifier()
    model.fit(X,y)
    predicted = model.predict(XX)
    return predicted
Label_pred_RandomForest = RFC(Feature_no_onehot_train,Label_no_onehot_train.values.ravel(),Feature_no_onehot_test)
print(accuracy_score(Label_no_onehot_test, Label_pred_RandomForest))
print(roc_auc_score(Label_no_onehot_test, Label_pred_RandomForest))

0.9355945121951219
0.5900970088924818


In [20]:
from sklearn.svm import SVC
def SVM(X,y,XX):
    model = SVC(c=5.0)
    model.fit(X,y)
    predicted = model.predict(XX)
    return predicted
Label_pred_SVM = RFC(Feature_train,Label_train.values.ravel(),Feature_test)
print(accuracy_score(Label_test, Label_pred_SVM))
print(roc_auc_score(Label_test, Label_pred_SVM))


0.9397865853658537
0.565436066842254


In [21]:
from sklearn.svm import SVC
def SVM(X,y,XX):
    model = SVC(c=5.0)
    model.fit(X,y)
    predicted = model.predict(XX)
    return predicted
Label_pred_SVM = RFC(Feature_no_onehot_train,Label_no_onehot_train.values.ravel(),Feature_no_onehot_test)
print(accuracy_score(Label_no_onehot_test, Label_pred_SVM))
print(roc_auc_score(Label_no_onehot_test, Label_pred_SVM))


0.9340701219512195
0.5830261385071409
